In [2]:
import pandas as pd
import numpy as np
import os
import scipy
import sqlite3
import tensorflow as tf
import string

In [3]:
curr_path = os.path.realpath('')
print(curr_path)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
conn.close()

/Users/noellin/Jupyter_Notebooks/loan_driving2


In [4]:
conn = sqlite3.connect("lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status = 'Charged Off';", conn)
#print(df)
conn.close()

In [5]:
ddict = df.to_dict()
print(ddict['loan_amnt'][7])
ddict['sub_grade'][7]
g = zip(ddict['loan_amnt'].values(),ddict['sub_grade'].values(),ddict['id'].values(),ddict['loan_status'].values())
print(len(list(g)))

15000.0
45248


In [6]:
# create our data set
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
# df = pd.read_sql_query("select loan_amnt,loan_status,sub_grade from loan where loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)';", conn)
#df_tf = pd.read_sql_query("select loan_amnt, term, int_rate, annual_inc, dti, home_ownership from loan where loan_status =='Fully Paid' OR loan_status == 'Charged Off' OR loan_status == 'Default'", conn)

###@@@@@ 
#         NORMALIZE and add loan_amnt and annual_inc back later 
###@@@@
df_tf = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, dti  from loan where emp_length != 'n/a' OR loan_status =='Fully Paid' OR loan_status == 'Charged Off' OR loan_status == 'Default'", conn)
# df_desc = pd.read_sql_query("select loan_amnt, loan_status, sub_grade from loan where desc != 'NULL' AND loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)'", conn)
print (len(df_tf))

data = []
results = []
for i in range(10000):  # data set of 10000 for faster run time. Replace with range(len(df_tf)) later
    # one hot encoding for loan status. Or 0, 1, 2
    # loan_status = ['1', '0', '0'] if df_tf.iloc[i][0] == 'Fully Paid' else ['0', '1', '0'] if df_tf.iloc[i][0] == 'Default' else ['0', '0', '1']
    loan_status = 0 if df_tf.iloc[i][0] == 'Fully Paid' else 1 # bad if not fully paid
    
    # 0 if term is 36 months, 1 if 60 months
    term = 0 if df_tf.iloc[i][1] == ' 36 months' else 1
    
    # change interest rate from str to float
    int_rate = float(df_tf.iloc[i][2].replace('%', ''))
    
    # get employment length
    txt = ''.join([c for c in df_tf.iloc[i][3].lower() if not c in set(string.punctuation)])
    emp_length_list = ([int(s) for s in txt.split() if s.isdigit()])
    emp_length = 0 if emp_length_list == [] else emp_length_list[0]

    # 0 if renting, 1 if owner
    ownership = 0 if df_tf.iloc[i][4] == 'RENT' else 1
    
    # verification status, 0 for not verified, 1 for source verified and 2 for verified
    #verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1 if df_tf.iloc[i][5] == 'Source Verified' else 2
    verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1

    # append in feature vector
    data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6]]) #df_tf.iloc[i][7], df_tf.iloc[i][8]])
                 
    # add loan status to results 
    results.append(loan_status)

print (data[-1:])
print (results[-1:])
X_train = data[:int(len(data)/2)]
y_train = results[:int(len(results)/2)]
X_val = data[int(len(data)/2): int(len(data)*0.75)]
y_val = results[int(len(results)/2): int(len(results)*0.75)]
X_test = data[int(len(data)*.75):]
y_test = results[int(len(results)*.75):]

conn.close()
# feature vector - predictor will predict ("charged off/default status") vs. 
# [1(maybe check this),'sub_grade','loan_amnt','emp_length','home_ownership','annual_inc','verification_status','dti'(debt-to-income),'']

852525
[[1, 13.99, 8, 1, 1, 9.6999999999999993]]
[0]


In [7]:
X_train_arr = np.array(X_train)
y_train_arr = np.array(y_train)
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train_arr)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[30,15], n_classes=2,feature_columns=feature_cols, label_keys=['0','1','2'])
# dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) 
dnn_clf.fit(X_train_arr, y_train_arr, batch_size=10, steps=5000)
# dnn_clf.fit(X_train_arr, y_train_arr)

ValueError: label_keys is not supported for n_classes=2.

In [ ]:
from sklearn.metrics import accuracy_score

X_val_arr = np.array(X_val)
y_val_arr = np.array(y_val)
y_pred = dnn_clf.predict_classes(X_val_arr, as_iterable=False)
#print (X_val_arr)
#print (X_val[0])
#print(y_pred)
print (accuracy_score(y_val_arr, y_pred))

In [1]:
X_train_arr = numpy.array(X_train)
y_train_arr = numpy.array(y_train)

print (y_train_arr[:5])
print (y_train[:5])
print (X_train_arr[:5])

NameError: name 'numpy' is not defined

In [ ]:
# for testing library purposes
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print (cancer['data'])
print (cancer['target'])